# Chapter 7: CI/CD Pipelines in the Cloud

Infrastructure as Code gives us repeatable, version-controlled environments. Now we must address how applications flow into those environments. Continuous Integration and Continuous Delivery (CI/CD) represent the automation of software delivery—from the moment a developer commits code to when that code serves production traffic.

In modern cloud computing, CI/CD is not merely a convenience; it is a competitive necessity. Organizations deploying multiple times per day (the elite performers identified in the DORA State of DevOps reports) achieve significantly lower change failure rates and faster recovery times than those deploying monthly. This chapter teaches you to build the pipelines that enable this velocity without sacrificing stability.

## 7.1 The CI/CD Philosophy: Automation of Trust

### Continuous Integration (CI)
CI is the practice of merging all developers' working copies to a shared mainline several times a day, validated by automated builds and tests.

**Core Principles:**
1.  **Trunk-Based Development:** Developers work in short-lived feature branches (hours, not days) merging frequently to `main`.
2.  **Automated Testing:** Every merge triggers unit tests, integration tests, and code quality checks.
3.  **Fast Feedback:** Failures notify developers within minutes, enabling immediate correction.

### Continuous Delivery vs. Continuous Deployment
**Continuous Delivery (CD):** Code is automatically built, tested, and *prepared* for release to production. The final deployment decision is manual (business approval, timing considerations).

**Continuous Deployment (CD):** Every change that passes all quality gates is automatically deployed to production without human intervention. This requires comprehensive automated testing and feature flags to manage risk.

**Formula for Deployment Frequency:**
$$ \text{Deployment Confidence} = \frac{\text{Automated Test Coverage} \times \text{Observability}}{\text{Manual Steps}} $$

As manual steps approach zero with high test coverage, deployment frequency increases exponentially.

## 7.2 Building a Pipeline: The Stages of Software Delivery

A production-grade pipeline consists of distinct stages, each a gate that must be passed before proceeding.

### Stage 1: Source (Trigger)
The pipeline initiates when code changes are detected:
*   **Git Events:** Push to `main`, pull request opened/updated, tag created.
*   **Webhook:** Git provider (GitHub, GitLab, Bitbucket) notifies CI service.
*   **Polling:** CI service checks repository periodically (less common now).

### Stage 2: Build (Compilation & Packaging)
Transforming source code into deployable artifacts:
*   **Compilation:** Java, C++, Go binaries.
*   **Packaging:** Docker image builds, JAR/WAR files, npm packages.
*   **Artifact Storage:** Immutable artifacts pushed to registries (ECR, ACR, GCR, Docker Hub, Nexus, Artifactory).

**Code Snippet: Multi-Stage Docker Build Optimization**
```dockerfile
# Build stage - includes dev dependencies
FROM node:18-alpine AS builder
WORKDIR /app
COPY package*.json ./
RUN npm ci
COPY . .
RUN npm run build

# Production stage - minimal attack surface
FROM node:18-alpine AS production
WORKDIR /app
COPY package*.json ./
RUN npm ci --only=production && \
    addgroup -g 1001 -S nodejs && \
    adduser -S nodejs -u 1001
COPY --from=builder --chown=nodejs:nodejs /app/dist ./dist
USER nodejs
EXPOSE 3000
HEALTHCHECK --interval=30s --timeout=3s \
  CMD curl -f http://localhost:3000/health || exit 1
CMD ["node", "dist/server.js"]
```

### Stage 3: Testing (Quality Gates)
Automated validation occurs in parallel or sequence:

**Static Application Security Testing (SAST):**
Analyze source code for vulnerabilities without execution.
*   Tools: SonarQube, Checkmarx, Semgrep, Bandit (Python), ESLint Security.
*   **Fail Threshold:** High/Critical vulnerabilities block deployment.

**Unit Testing:**
Fast, isolated tests (milliseconds) mocking dependencies.
*   Target: >80% code coverage for critical paths.
*   Tools: Jest (JavaScript), JUnit (Java), pytest (Python).

**Integration Testing:**
Validate component interactions (databases, APIs).
*   Use test containers (ephemeral Docker databases) for isolation.
*   Test database migrations (schema changes) in this stage.

**Code Snippet: Integration Test with Testcontainers**
```python
# test_database.py
import pytest
from testcontainers.postgres import PostgresContainer
from sqlalchemy import create_engine

def test_database_connection():
    with PostgresContainer("postgres:15-alpine") as postgres:
        engine = create_engine(postgres.get_connection_url())
        with engine.connect() as connection:
            result = connection.execute("SELECT version()")
            assert result.fetchone() is not None
```

### Stage 4: Security Scanning (DevSecOps)
Beyond SAST, comprehensive security validation:

**Software Composition Analysis (SCA):**
Scan dependencies for known vulnerabilities (CVEs).
*   Tools: Snyk, OWASP Dependency-Check, GitHub Dependabot.
*   Example: Detecting that `log4j-core-2.14.0.jar` contains Log4Shell (CVE-2021-44228).

**Container Scanning:**
Analyze Docker images for OS vulnerabilities and misconfigurations.
```bash
# Trivy container scan example
trivy image --severity HIGH,CRITICAL --exit-code 1 myapp:latest

# Check for secrets in code (prevent credential leakage)
truffleHog --regex --entropy=False .
```

**Dynamic Application Security Testing (DAST):**
Test running application for vulnerabilities (SQL injection, XSS).
*   Executed in staging environment against deployed application.
*   Tools: OWASP ZAP, Burp Suite.

### Stage 5: Deployment (Release)
Promoting artifacts through environments:

**Environment Promotion Strategy:**
Artifacts should be immutable—build once, deploy the same binary to dev, staging, and production. Configuration (environment variables) changes per environment, not the code.

```
Build → Artifact Registry → Dev Deploy → Staging Deploy → Prod Deploy
                ↑                    ↑              ↑            ↑
             Same Image           Same Image    Same Image   Same Image
             Different Config     Different Config, etc.
```

## 7.3 Cloud-Native CI/CD Tools

Each major cloud provider offers integrated CI/CD services that minimize setup and maximize integration with cloud resources.

### AWS CodePipeline + CodeBuild
Fully managed continuous delivery service that automates release pipelines.

**Architecture:**
*   **CodePipeline:** Orchestration engine defining workflow stages.
*   **CodeBuild:** Serverless build environment (compiles, tests, packages).
*   **CodeDeploy:** Automates deployment to EC2, Lambda, ECS, or on-premises.

**Code Snippet: AWS CodeBuild Specification (`buildspec.yml`)**
```yaml
version: 0.2

env:
  variables:
    AWS_DEFAULT_REGION: us-east-1
    IMAGE_REPO_NAME: my-app
    IMAGE_TAG: latest
  secrets-manager:
    DOCKER_HUB_TOKEN: prod/dockerhub:token  # Secure credential retrieval

phases:
  pre_build:
    commands:
      - echo Logging in to Amazon ECR...
      - aws ecr get-login-password --region $AWS_DEFAULT_REGION | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com
      - echo Running security scans...
      - checkov --file infrastructure/
  
  build:
    commands:
      - echo Build started on `date`
      - echo Building the Docker image...
      - docker build -t $IMAGE_REPO_NAME:$IMAGE_TAG .
      - docker tag $IMAGE_REPO_NAME:$IMAGE_TAG $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$CODEBUILD_RESOLVED_SOURCE_VERSION
      
  post_build:
    commands:
      - echo Build completed on `date`
      - echo Pushing the Docker image...
      - docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$CODEBUILD_RESOLVED_SOURCE_VERSION
      - printf '{"ImageUri":"%s"}' $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$CODEBUILD_RESOLVED_SOURCE_VERSION > imageDetail.json

artifacts:
  files:
    - imageDetail.json
    - infrastructure/**/*
  name: build-$(date +%Y-%m-%d-%H-%M-%S)

cache:
  paths:
    - '/root/.m2/**/*'
    - '/root/.npm/**/*'
```

### Azure DevOps (Pipelines)
Comprehensive DevOps platform with Azure Pipelines for CI/CD.

**Key Features:**
*   **YAML-Based:** Pipeline as code stored in repository (`azure-pipelines.yml`).
*   **Multi-Cloud:** Can deploy to AWS and GCP, not just Azure.
*   **Self-Hosted Agents:** Use your own build machines when cloud-hosted agents are insufficient.

**Code Snippet: Azure Pipeline for Kubernetes Deployment**
```yaml
# azure-pipelines.yml
trigger:
  - main

pool:
  vmImage: 'ubuntu-latest'

variables:
  dockerRegistryServiceConnection: 'myACR'
  imageRepository: 'myapp'
  containerRegistry: 'myregistry.azurecr.io'
  dockerfilePath: '$(Build.SourcesDirectory)/Dockerfile'
  tag: '$(Build.BuildId)'
  
  # Agent VM image name
  vmImageName: 'ubuntu-latest'

stages:
- stage: Build
  displayName: Build and push stage
  jobs:
  - job: Build
    displayName: Build
    pool:
      vmImage: $(vmImageName)
    steps:
    - task: Docker@2
      displayName: Build and push image
      inputs:
        command: buildAndPush
        repository: $(imageRepository)
        dockerfile: $(dockerfilePath)
        containerRegistry: $(dockerRegistryServiceConnection)
        tags: |
          $(tag)
          latest

- stage: Deploy
  displayName: Deploy to AKS
  dependsOn: Build
  condition: succeeded()
  jobs:
  - deployment: Deploy
    displayName: Deploy to Kubernetes
    environment: 'production.production-namespace'
    pool:
      vmImage: $(vmImageName)
    strategy:
      runOnce:
        deploy:
          steps:
          - task: KubernetesManifest@0
            displayName: Deploy to Kubernetes cluster
            inputs:
              action: deploy
              manifests: |
                $(Pipeline.Workspace)/manifests/deployment.yml
                $(Pipeline.Workspace)/manifests/service.yml
              containers: |
                $(containerRegistry)/$(imageRepository):$(tag)
```

### Google Cloud Build
Serverless CI/CD platform deeply integrated with GCP services.

**Distinctive Features:**
*   **Build Config as Code:** `cloudbuild.yaml` defines steps using containerized commands.
*   **Buildpacks:** Automatic containerization without Dockerfile (detects language and builds optimized image).
*   **Integration with Binary Authorization:** Cryptographic signing of images before deployment to GKE.

**Code Snippet: Cloud Build Configuration**
```yaml
# cloudbuild.yaml
steps:
  # Build container image
  - name: 'gcr.io/cloud-builders/docker'
    args: ['build', '-t', 'gcr.io/$PROJECT_ID/myapp:$COMMIT_SHA', '.']
  
  # Push to Container Registry
  - name: 'gcr.io/cloud-builders/docker'
    args: ['push', 'gcr.io/$PROJECT_ID/myapp:$COMMIT_SHA']
  
  # Run security scan using Container Analysis
  - name: 'gcr.io/cloud-builders/gcloud'
    entrypoint: 'bash'
    args:
      - '-c'
      - |
        echo "Scanning for vulnerabilities..."
        gcloud artifacts docker images scan \
          gcr.io/$PROJECT_ID/myapp:$COMMIT_SHA \
          --format='value(response.scan)'
  
  # Deploy to Cloud Run
  - name: 'gcr.io/cloud-builders/gcloud'
    args:
      - 'run'
      - 'deploy'
      - 'myapp-service'
      - '--image'
      - 'gcr.io/$PROJECT_ID/myapp:$COMMIT_SHA'
      - '--region'
      - 'us-central1'
      - '--platform'
      - 'managed'
      - '--allow-unauthenticated'

images:
  - 'gcr.io/$PROJECT_ID/myapp:$COMMIT_SHA'

options:
  logging: CLOUD_LOGGING_ONLY
```

### Jenkins: The Open Source Standard
While cloud-native tools offer convenience, Jenkins remains ubiquitous in enterprise environments due to its plugin ecosystem (1800+ plugins) and flexibility.

**Modern Jenkins: Pipeline as Code (Jenkinsfile)**
```groovy
// Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    
    environment {
        DOCKER_REGISTRY = 'myregistry.azurecr.io'
        IMAGE_NAME = 'myapp'
        DOCKER_CREDENTIALS = credentials('docker-registry-credentials')
    }
    
    stages {
        stage('Checkout') {
            steps {
                checkout scm
            }
        }
        
        stage('Build') {
            steps {
                sh 'npm ci'
                sh 'npm run build'
            }
        }
        
        stage('Test') {
            parallel {
                stage('Unit Tests') {
                    steps {
                        sh 'npm test'
                    }
                    post {
                        always {
                            publishTestResults testResultsPattern: 'test-results.xml'
                        }
                    }
                }
                stage('Lint') {
                    steps {
                        sh 'npm run lint'
                    }
                }
            }
        }
        
        stage('Security Scan') {
            steps {
                sh 'trivy filesystem --exit-code 1 --severity HIGH .'
            }
        }
        
        stage('Build Docker Image') {
            steps {
                script {
                    dockerImage = docker.build("${DOCKER_REGISTRY}/${IMAGE_NAME}:${BUILD_NUMBER}")
                }
            }
        }
        
        stage('Push Image') {
            steps {
                script {
                    docker.withRegistry("https://${DOCKER_REGISTRY}", DOCKER_CREDENTIALS) {
                        dockerImage.push()
                        dockerImage.push('latest')
                    }
                }
            }
        }
        
        stage('Deploy to Staging') {
            when {
                branch 'develop'
            }
            steps {
                sh 'kubectl config use-context staging'
                sh "kubectl set image deployment/myapp myapp=${DOCKER_REGISTRY}/${IMAGE_NAME}:${BUILD_NUMBER}"
            }
        }
    }
    
    post {
        failure {
            emailext (
                subject: "FAILED: Job '${env.JOB_NAME} [${env.BUILD_NUMBER}]'",
                body: "Check console output at ${env.BUILD_URL}",
                to: "${env.CHANGE_AUTHOR_EMAIL}"
            )
        }
    }
}
```

## 7.4 Container-Based Pipelines and GitOps

### Kubernetes-Native CI/CD
For organizations running Kubernetes, traditional push-based deployment (CI system executes `kubectl apply`) creates security and operational challenges. **GitOps** inverts this model.

**GitOps Principles (as defined by Weaveworks):**
1.  **Declarative:** System state defined in Git (YAML manifests, Helm charts, Kustomize).
2.  **Versioned & Immutable:** Git is the single source of truth; rollback via `git revert`.
3.  **Pulled Automatically:** Agents in the cluster (ArgoCD, Flux) pull desired state and apply it.
4.  **Continuously Reconciled:** Agents detect drift (manual changes via `kubectl`) and self-heal to match Git.

**ArgoCD Example:**
```yaml
# Application definition stored in Git
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: myapp-production
  namespace: argocd
spec:
  project: default
  source:
    repoURL: https://github.com/org/gitops-repo.git
    targetRevision: HEAD
    path: overlays/production
    helm:
      valueFiles:
        - values-production.yaml
  destination:
    server: https://kubernetes.default.svc
    namespace: production
  syncPolicy:
    automated:
      prune: true        # Remove resources not in Git
      selfHeal: true     # Override manual changes
    syncOptions:
      - CreateNamespace=true
```

**CI vs. GitOps Separation:**
*   **CI (Build):** Builds image, runs tests, pushes to registry, updates Git repo with new image tag.
*   **GitOps (Deploy):** ArgoCD detects Git change, pulls and applies manifests to cluster.

## 7.5 Deployment Strategies: Minimizing Risk

Not all deployments are equal. Different strategies balance risk, resource cost, and complexity.

### Rolling Deployment (Default)
Gradually replaces old instances with new ones.
*   **Process:** Take one instance offline, deploy new version, health check, proceed to next.
*   **Pros:** No extra resources required.
*   **Cons:** Rollback is slow; mixed versions run simultaneously (backward compatibility required).

**Kubernetes Rolling Update:**
```yaml
spec:
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 25%        # Max pods above desired count
      maxUnavailable: 25%  # Max pods unavailable during update
```

### Blue/Green Deployment
Two identical environments ("Blue" = current, "Green" = new). Traffic instantly switches from Blue to Green after testing.
*   **Pros:** Instant rollback (switch back to Blue), zero downtime, complete environment validation before traffic hits it.
*   **Cons:** Double resource requirements during deployment.

**Implementation:**
```bash
# Route 53 or ALB weight-based routing
# 1. Deploy Green environment
kubectl apply -f deployment-green.yaml

# 2. Verify Green health
kubectl run -it --rm debug --image=curlimages/curl --restart=Never -- http://green-service/health

# 3. Switch ALB target group from Blue to Green
aws elbv2 modify-target-group --target-group-arn $GREEN_TG --health-check-path /health

# 4. If issues detected, immediate rollback by switching back to Blue
```

### Canary Deployment
Route small percentage of traffic (e.g., 5%) to new version, monitor metrics, gradually increase to 100%.
*   **Pros:** Real-world testing with limited blast radius; automatic rollback if error rate increases.
*   **Cons:** Complex to implement; requires sophisticated traffic splitting and monitoring.

**Istio Service Mesh Example:**
```yaml
apiVersion: networking.istio.io/v1beta1
kind: VirtualService
metadata:
  name: myapp
spec:
  hosts:
    - myapp.example.com
  http:
    - route:
        - destination:
            host: myapp
            subset: v1
          weight: 95
        - destination:
            host: myapp
            subset: v2
          weight: 5
      # Automatic rollback condition
      retries:
        attempts: 3
        perTryTimeout: 2s
```

## 7.6 Secrets Management in Pipelines

Hardcoding credentials in pipeline configurations is a critical security anti-pattern.

**Best Practices:**
1.  **Native Secret Stores:** Use AWS Secrets Manager, Azure Key Vault, or GCP Secret Manager.
2.  **Pipeline Integration:** Inject secrets as environment variables at runtime, never log them.
3.  **Dynamic Credentials:** Use temporary credentials (AWS STS, Azure Managed Identity) rather than long-lived keys.

**Code Snippet: Secure Secret Handling**
```yaml
# GitHub Actions example
name: Deploy
on:
  push:
    branches: [main]

jobs:
  deploy:
    runs-on: ubuntu-latest
    permissions:
      id-token: write  # Required for OIDC
      contents: read
    
    steps:
      - uses: actions/checkout@v4
      
      # AWS OIDC authentication (no long-lived keys stored)
      - name: Configure AWS Credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: arn:aws:iam::123456789:role/GitHubActionsRole
          aws-region: us-east-1
      
      # Retrieve secrets from AWS Secrets Manager
      - name: Get DB credentials
        uses: aws-actions/aws-secretsmanager-get-secrets@v1
        with:
          secret-ids: |
            prod/myapp/database
          parse-json-secrets: true
      
      # Use secrets (masked in logs automatically)
      - name: Deploy
        run: |
          helm upgrade --install myapp ./chart \
            --set database.password="${{ env.PROD_MYAPP_DATABASE_PASSWORD }}"
```

---

### Summary

In this chapter, we architected the automation backbone of cloud-native software delivery. You learned that CI/CD separates the mechanics of delivery from manual execution, enabling velocity through automation while maintaining quality through rigorous testing gates. We explored the complete pipeline lifecycle—from source control triggers through multi-stage builds, comprehensive security scanning (SAST, DAST, SCA), and artifact promotion. You mastered cloud-native tooling across AWS CodePipeline, Azure DevOps, and Google Cloud Build, while understanding Jenkins' continued relevance in enterprise environments. We examined deployment strategies that minimize risk: rolling updates for simplicity, Blue/Green for instant rollback capability, and Canary releases for real-world validation with limited exposure. Finally, we established security best practices including OIDC authentication, secrets management, and the principle of never persisting credentials in code or configuration.

With CI/CD pipelines operational, your infrastructure provisions automatically and your applications deploy seamlessly. However, modern cloud-native applications rarely run directly on virtual machines—they run in containers orchestrated by Kubernetes. Understanding how to build, deploy, and manage containerized workloads at scale is essential for any cloud practitioner.

**Next Up: Chapter 8 - Container Orchestration with Kubernetes**
In the next chapter, we will dive deep into the de facto standard for container orchestration. You will learn Kubernetes architecture (control plane, nodes, pods, services), deployment patterns for stateless and stateful applications, scaling mechanisms, and service mesh integration. We will deploy a microservice application to a managed Kubernetes cluster (EKS, AKS, or GKE), implementing health checks, rolling updates, and autoscaling policies that bring together everything you have learned about infrastructure, application design, and automated deployment.